# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys2 import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_csv = "/Users/laurelwilliamson/Desktop/GITHUB/python-api-challenge/WeatherPy/output_weather/new_weather.csv"


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
weather_df = pd.read_csv(weather_csv)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
nice_weather_df = weather_df.loc[(weather_df["Max Temp"] < 80) &
                                           (weather_df['Max Temp'] > 70)]

nice_weather_df = nice_weather_df.loc[nice_weather_df["Wind Speed"] < 10]
                                            
nice_weather_df = nice_weather_df.loc[nice_weather_df["Cloudiness"] == 0]
                                        
nice_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = nice_weather_df

hotel_df['Hotel Name'] = ""

params = {'radius': 5000, 'types': 'lodging', 'key': g_key, 'keyword': 'hotel'}

In [9]:
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    
    
    google_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(google_url, params=params).json()
    
    params['location'] = f'{latitude},{longitude}'
    
    hotel_result = response['results']
   
    
    try:
        hotel_name = hotel_result[0]['name']
        print(f'The closest hotel to {city} is {hotel_name}.')
        #print(f"The Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_result[0]['name']

    except:
        print("Cannot find data...Skipping.")
        
    print('---------------------------')
    


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map

rating = weather_df['Humidity'].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


fig = gmaps.figure(map_type="HYBRID")
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure


TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.